# Figure S1: Glacier projections by hydrological zones

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr 
import geopandas as gpd
    
import os
from glob import glob
from tqdm import tqdm
from tqdm.notebook import tqdm

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

os.chdir('/home/rooda/OGGM_results/')

import warnings
warnings.simplefilter("ignore")

In [ ]:
def postprocessing(ds, scenario): # clean dataframe
    ds = ds.to_dataframe()
    ds["scenario"] = scenario
    ds = ds.set_index("scenario", append=True)
    ds = ds.reorder_levels(['scenario', 'rgi_id', 'time'])
    return ds

## Data

In [ ]:
basins = gpd.read_file("zip:////home/rooda/Dropbox/Patagonia/MS2 Results/zenodo/basins_boundaries.zip")[["basin_id", "basin_zone"]].set_index("basin_id")

# historical
ts_hist = xr.open_dataset("/home/rooda/OGGM_results/runs/OGGM_historical.nc")[["volume", "area"]]
ids     = basins[basins.index.isin(ts_hist.rgi_id.to_pandas().tolist())]
ts_hist = ts_hist.assign_coords(rgi_id = ids.basin_zone.tolist())
ts_hist = ts_hist.groupby('rgi_id').sum()

ts_hist["smb"] = (ts_hist.volume.diff(dim = "time") * 900 / ts_hist.area) 
ts_hist_var  = postprocessing(ts_hist.std(dim="options"),  "historical")
ts_hist_mean = postprocessing(ts_hist.mean(dim="options"), "historical")

In [ ]:
# future
scenarios        = ["ct_random", "ssp126", "ssp245", "ssp370", "ssp585"]

ts_future_mean = []
ts_future_var  = []

for scenario in tqdm(scenarios): 
    ts_future_ssp   = xr.open_dataset("/home/rooda/OGGM_results/runs/OGGM_future_{}.nc".format(scenario))[["volume", "area"]]
    ids             = basins[basins.index.isin(ts_future_ssp.rgi_id.to_pandas().tolist())]
    ts_future_ssp   = ts_future_ssp.assign_coords(rgi_id = ids.basin_zone.tolist())
    ts_future_ssp   = ts_future_ssp.groupby('rgi_id').sum()    
    ts_future_ssp["smb"] = (ts_future_ssp.volume.diff(dim = "time") * 900 / ts_future_ssp.area) 
    
    ts_future_var.append(postprocessing(ts_future_ssp.std(dim="options"),   scenario))
    ts_future_mean.append(postprocessing(ts_future_ssp.mean(dim="options"), scenario))

ts_future_var  = pd.concat(ts_future_var)
ts_future_mean = pd.concat(ts_future_mean)

In [ ]:
# concat historical and future perios
ts_var  = pd.concat([ts_hist_var,  ts_future_var]).reset_index()
ts_mean = pd.concat([ts_hist_mean, ts_future_mean]).reset_index()

dict_replace = {"scenario": {"historical":'Historical', 
                             "ct_random":'Commitment run', 
                             "ssp126":'SSP 1-2.6', 
                             "ssp245":'SSP 2-4.5',
                             "ssp370":'SSP 3-7.0', 
                             "ssp585":'SSP 5-8.5'}}

ts_var      = ts_var.replace(dict_replace)
ts_mean     = ts_mean.replace(dict_replace)
ts_mean_ref = ts_mean[ts_mean.time == 2020]

## Plot

In [ ]:
cl = px.colors.colorbrewer.RdYlBu

scenarios   = ["Historical", "Commitment run", "SSP 1-2.6", "SSP 2-4.5", "SSP 3-7.0", "SSP 5-8.5"]
scen_colors = {"Historical":"rgba(0, 0, 0, 0.8)", 
               "Commitment run":"rgba(0, 0, 0, 0.5)", 
               "SSP 1-2.6":cl[9], 
               "SSP 2-4.5":cl[8], 
               "SSP 3-7.0":cl[3],
               "SSP 5-8.5":cl[1]}

shaded_colors = {"Historical":"rgba(0, 0, 0, 0.1)", 
                 "SSP 1-2.6": "rgba(49,54,149, 0.1)", 
                 "SSP 5-8.5": "rgba(215,48,39,0.1)"}

basins_id = ['PPY', 'PCA','NPI-E','NPI-W','SPI-N', 'SPI-C', 'SPI-S', 'GCN', 'CDI']

fig    = make_subplots(rows=9, cols=3, horizontal_spacing = 0.04, vertical_spacing = 0.01, shared_xaxes= True, shared_yaxes= False, 
                       row_titles = basins_id, subplot_titles= ["Glacier volume (%)", "Glacier area (%)", "Specific mass balance (kg m<sup>-2</sup>)"])

for x in range(0,9):
    for y in range(0,3):
        for t in range(0,6):
             
            # time series for each subplot and scenario
            time_series_id    = ts_mean[ts_mean.rgi_id == basins_id[x]][ts_mean.scenario == scenarios[t]]
            time_series_sd_id = ts_var[ts_var.rgi_id == basins_id[x]][ts_var.scenario == scenarios[t]]    
            ts_ref_id         = ts_mean_ref[ts_mean_ref.rgi_id == basins_id[x]]
            
            if x==0 and y==0: # legend only for first plot
        
                fig.add_trace(go.Scatter(x=time_series_id.time, y=time_series_id.iloc[:,y+3]/ts_ref_id.iloc[0,y+3], 
                                         mode='lines', name= scenarios[t], 
                                         line=dict(color=scen_colors[scenarios[t]], width = 1.5), showlegend=True, legendgroup=t), row=x+1, col=y+1)
                
                if t == 0 or t == 2 or t >= 5: # uncertainty: +-1 sd
                    fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]+time_series_sd_id.iloc[:,y+3])/ts_ref_id.iloc[0,y+3], 
                                             line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], showlegend=False, legendgroup='g1'), row=x+1, col=y+1)
                    fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]-time_series_sd_id.iloc[:,y+3])/ts_ref_id.iloc[0,y+3], 
                                             line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], fill='tonexty', showlegend=False, legendgroup=t), row=x+1, col=y+1)

            else:
                if y==2: # dont normalize specific mass balance
            
                    fig.add_trace(go.Scatter(x=time_series_id.time, y=time_series_id.iloc[:,y+3], mode='lines', name= scenarios[t], 
                                             line=dict(color=scen_colors[scenarios[t]], width = 1.5), showlegend=False, legendgroup=t), row=x+1, col=y+1)

                    if t == 0 or t == 2 or t >= 5: # uncertainty: +-1 sd
                        fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]+time_series_sd_id.iloc[:,y+3]), 
                                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], showlegend=False, legendgroup=t), row=x+1, col=y+1)
                        fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]-time_series_sd_id.iloc[:,y+3]), 
                                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], fill='tonexty', showlegend=False, legendgroup=t), row=x+1, col=y+1)

                else:
                    # mean value
                    fig.add_trace(go.Scatter(x=time_series_id.time, y=time_series_id.iloc[:,y+3]/ts_ref_id.iloc[0,y+3], mode='lines', name= scenarios[t], 
                                             line=dict(color=scen_colors[scenarios[t]], width = 1.5), showlegend=False, legendgroup=t), row=x+1, col=y+1)

                    
                    if t == 0 or t == 2 or t >= 5: # uncertainty: +-1 sd
                        fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]+time_series_sd_id.iloc[:,y+3])/ts_ref_id.iloc[0,y+3], 
                                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], showlegend=False, legendgroup=t), row=x+1, col=y+1)
                        fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]-time_series_sd_id.iloc[:,y+3])/ts_ref_id.iloc[0,y+3], 
                                                 line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], fill='tonexty', showlegend=False, legendgroup=t), row=x+1, col=y+1)

#  some tweaks
for x in range(0,9):
    for y in range(0,3):                
        if y==2: fig.update_yaxes(dtick = 2000,  row=x+1, col=y+1)
        else: fig.update_yaxes(range = [0, 1.2], dtick = 0.5, tickformat=".0%", row=x+1, col=y+1)
        
fig.update_yaxes(range = [-5500, 500], dtick = 3000, row=5, col=3)

fig.update_yaxes(ticks="outside", zeroline=False, griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", showline = True, linecolor = 'black', linewidth = 0.2, mirror=True, tickangle = -90)
fig.update_xaxes(ticks="outside", zeroline=False, griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", showline = True, linecolor = 'black', linewidth = 0.2, mirror=True, dtick = 20)
fig.update_layout(legend=dict(yanchor="top", y=-0.02, xanchor="left", x=0.15, orientation="h", bgcolor = 'rgba(0,0,0,0.0)'))
fig.update_layout(height=1200, width=1050, template = "seaborn", margin = dict(l=20, r=20, b=30, t=30), hovermode = False)

# save figure 
fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_S1_Glacier_projections.png", scale=4)
fig.show()

## Text

In [ ]:
# regional smb
ts_future_ssp   = xr.open_mfdataset("/home/rooda/OGGM_results/runs/OGGM_future_ssp*.nc")[["volume", "area"]]
ts_future_ssp   = ts_future_ssp.sel(time = slice(2070,2100))
ids             = basins[basins.index.isin(ts_future_ssp.rgi_id.to_pandas().tolist())]
ts_future_ssp   = ts_future_ssp.assign_coords(rgi_id = ids.basin_zone.tolist())
ts_future_ssp   = ts_future_ssp.groupby('rgi_id').sum()    
ts_future_ssp["smb"] = (ts_future_ssp.volume.diff(dim = "time") * 900 / ts_future_ssp.area) 
ts_future_ssp   = ts_future_ssp.smb.load()

"the mean specific mass balance in NPI-E ranged from {} ± {} kg m-2 in SSP1-2.6 to {} ± {} kg m-2 in SSP 5-8.5".format(
    int(ts_future_ssp.sel(options = "ssp126").sel(rgi_id = "NPI-E").mean(dim = "time").mean(dim = "options")),
    int(ts_future_ssp.sel(options = "ssp126").sel(rgi_id = "NPI-E").mean(dim = "time").std(dim = "options")),
    int(ts_future_ssp.sel(options = "ssp585").sel(rgi_id = "NPI-E").mean(dim = "time").mean(dim = "options")),
    int(ts_future_ssp.sel(options = "ssp585").sel(rgi_id = "NPI-E").mean(dim = "time").std(dim = "options")))

In [ ]:
# is the maximum between the scenarios?
pd.concat([ts_future_ssp.sel(options = "ssp126").mean(dim = "time").mean(dim = "options").to_dataframe(),
           ts_future_ssp.sel(options = "ssp585").mean(dim = "time").mean(dim = "options").to_dataframe()], axis = 1)